In [ ]:
import pandas as pd
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import torch
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Inicializar spaCy
nlp = spacy.load("en_core_web_sm")


# Cargar el modelo BERT y el tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Asegurarse de que el modelo esté en modo de evaluación
model.eval()

# Ruta al archivo JSON (ajusta el path según donde esté ubicado el archivo)
path_file = "full_format_recipes.json"

# Cargar el archivo JSON como un DataFrame de Pandas
def load_recipes(file_path):
    try:
        with open(file_path, "r") as file:
            recipes_df = pd.read_json(file)
            recipes_df.dropna(inplace=True)
            print("Datos cargados exitosamente.")
            return recipes_df
    except ValueError as e:
        print(f"Error al leer el archivo JSON: {e}")
        return None

def preprocess_text_spacy(text):

    # Eliminar caracteres no alfabéticos
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Procesar el texto con SpaCy
    doc = nlp(text)

    # Tokenización y Lematización con SpaCy
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    text_lemma = ' '.join(tokens)

    # Convertimos el texto en una lista de tokens
    words = re.findall(r'\S+', text_lemma)

    # Devolver el texto procesado como lista de palabras
    return words

def compute_tfidf(docs):

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(docs)
    return tfidf_matrix, vectorizer

def compute_word2vec(docs):

    # Tokenización de documentos
    tokenized_docs = [doc.split() for doc in docs]

    # Entrenamiento de Word2Vec
    model = Word2Vec(sentences=tokenized_docs, vector_size=100, window=5, min_count=1, workers=4)

    # Calcular promedio de embeddings para cada documento
    doc_vectors = []
    for tokens in tokenized_docs:
        word_vectors = [model.wv[word] for word in tokens if word in model.wv]
        if word_vectors:
            doc_vectors.append(np.mean(word_vectors, axis=0))
        else:
            doc_vectors.append(np.zeros(model.vector_size))

    return np.array(doc_vectors), model


def compute_bert_embeddings(docs, batch_size=16): # Usa un bucle que procese
    embeddings = []                                     # los documentos en lotes pequeños (por ejemplo, de 16 o 32 documentos a la vez).
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)


# Cargar recetas
recipes_df = load_recipes(path_file)

# Tomar una muestra de las primeras 20 filas para prueba
recipes_df = recipes_df.iloc[0:100, :]

# Obtener la columna 'desc' (descripción de las recetas)
text_to_process = recipes_df.loc[:, "desc"]
ratings = recipes_df.loc[:, "rating"]

# Aplicar el preprocesamiento de texto a cada descripción
processed_words = text_to_process.apply(preprocess_text_spacy)

# Convertir la lista de palabras procesadas a un formato adecuado para TF-IDF y Word2Vec (como lista de strings)
processed_docs = [' '.join(words) for words in processed_words]

# 1. Aplicar TF-IDF
tfidf_matrix, vectorizer = compute_tfidf(processed_docs)

terms = vectorizer.get_feature_names_out()
matrix=tfidf_matrix.toarray()




# 2. Aplicar Word2Vec
word2vec_vectors, word2vec_model = compute_word2vec(processed_docs)
#print("Vectores promedio de Word2Vec:")
#print(word2vec_vectors)

plt.figure(2)
# Plotear los vectores
plt.scatter(word2vec_vectors[:, 0], word2vec_vectors[:, 1])
plt.title("Representación de recetas con Word2Vec (PCA)")
plt.show()

# 3. Aplicar BERT
# Convertir las descripciones a una lista de documentos (sin preprocesar)
processed_docs = text_to_process.tolist()

# Obtener los embeddings de BERT para las descripciones
bert_embeddings = compute_bert_embeddings(processed_docs)
#Verificar dimensión de la matriz de embeddings.
print(bert_embeddings.shape)  # (batch_size, embedding_dim)
# Guardar embeddings en un archivo
np.save("bert_embeddings.npy", bert_embeddings)

# Cargar embeddings desde el archivo
#bert_embeddings = np.load("bert_embeddings.npy")

"""
# Mostrar los embeddings de BERT para los primeros 3 documentos
print("Embeddings de BERT:")
print(bert_embeddings[:3])  # Muestra los primeros 3 embeddings
"""



#######################################
#Redes Neuronales utilizando PyTorch
#######################################


import torch.nn as nn
import torch.optim as optim

# 4. Entrenamiento y evaluaci´on de modelos de regresión

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(bert_embeddings, ratings, test_size=0.2, random_state=42)

# Estandarizar los datos (si es necesario para KNN, ya que KNN depende de las distancias entre puntos)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#####
###  Regresor K-NN  ###
#####

def KNN_Regresor(X_train_scaled,X_test_scaled,y_train,n_neighbors=10):

    # Entrenamiento del modelo KNN Regressor
    knn_model = KNeighborsRegressor(n_neighbors)  # Puedes ajustar 'n_neighbors' según sea necesario
    knn_model.fit(X_train_scaled, y_train)

    # Predicción y evaluación del modelo
    y_pred_knn = knn_model.predict(X_test_scaled)

    # Calcular el error cuadrático medio (MSE) para KNN
    mse_knn = mean_squared_error(y_test, y_pred_knn)
    print(f"KNN Regressor MSE: {mse_knn}")

    # Calcular mae para KNN
    mae_knn = mean_absolute_error(y_test, y_pred_knn)

    print(f"KNN Regressor MAE: {mae_knn}")

    return y_pred_knn

y_pred_knn=KNN_Regresor(X_train_scaled,X_test_scaled,y_train,n_neighbors=5)

######
###  Regresión_NN con pytorch ###
######
# Definir el modelo de red neuronal
class RegressionNN(nn.Module):
    def __init__(self, input_size):
        super(RegressionNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)  # Capa de entrada
        self.fc2 = nn.Linear(128, 64)  # Capa oculta
        self.fc3 = nn.Linear(64, 1)  # Capa de salida (valor de rating)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Convertir los datos a tensores de PyTorch
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Inicializar el modelo
model_nn = RegressionNN(input_size=X_train_scaled.shape[1])

# Definir la función de pérdida y el optimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(model_nn.parameters(), lr=1e-3)

# Entrenamiento de la red neuronal
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model_nn(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass y optimización
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #if (epoch+1) % 10 == 0:
        # print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

# Predicción y evaluación del modelo
model_nn.eval()  # Poner el modelo en modo evaluación
with torch.no_grad():
    y_pred_nn = model_nn(X_test_tensor)

# Calcular el error cuadrático medio (MSE) para la red neuronal
mse_nn = mean_squared_error(y_test_tensor.numpy(), y_pred_nn.numpy())
print(f"Neural Network MSE: {mse_nn}")
# Calcular mae para NN
mae_nn = mean_absolute_error(y_test_tensor.numpy(), y_pred_nn.numpy())
print(f"Neural Network MAE: {mae_nn}")




plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred_knn, color='blue', label="KNN Predictions", alpha=0.6)
plt.scatter(y_test, y_pred_nn, color='red', label="NN Predictions", alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='green', linestyle='--', label="Ideal Fit")
plt.xlabel("True Ratings")
plt.ylabel("Predicted Ratings")
plt.title("KNN vs Neural Network Predictions")
plt.legend()
plt.show()


#####
#5. Comparación KNN, NN y Fine-Tuning BER
#####


# Suponiendo que ya tienes los embeddings de BERT precalificados en 'bert_embeddings.npy'
# Cargar los embeddings de BERT
bert_embeddings = np.load("bert_embeddings.npy")

# Cargar las calificaciones (ratings)
ratings = recipes_df.loc[:, "rating"].values

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(bert_embeddings, ratings, test_size=0.2, random_state=42)

# Estandarizar los datos (es necesario para KNN y regresores)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convertir los datos a tensores de PyTorch
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Definir el modelo de regresión (una simple red neuronal con una capa densa)
class FineTunedBERT(nn.Module):
    def __init__(self, input_dim):
        super(FineTunedBERT, self).__init__()
        self.dense1 = nn.Linear(input_dim, 256)  # Capa de entrada
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(256, 1)  # Capa de salida (valor de rating)

    def forward(self, x):
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        return x

# Inicializar el modelo
input_dim = X_train_scaled.shape[1]  # Número de características de los embeddings de BERT
model = FineTunedBERT(input_dim)

# Definir la función de pérdida y el optimizador
criterion = nn.MSELoss()  # Usamos MSELoss para una tarea de regresión
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Entrenamiento del modelo
num_epochs = 100
for epoch in range(num_epochs):
    model.train()

    optimizer.zero_grad()
    output = model(X_train_tensor)

    loss = criterion(output, y_train_tensor)
    loss.backward()

    optimizer.step()

    #if (epoch + 1) % 5 == 0:
        #print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluar el modelo
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)
    y_pred = y_pred_tensor.numpy()

# Evaluación del modelo
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Fine-tunning MSE: {mse}")
print(f"Fine-tunning MAE: {mae}")

# Gráfico comparativo
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='green', alpha=0.6, label="Fine-Tuning BERT Predictions")
plt.plot(y_test, y_test, color='black', linestyle='--', label="Perfect Prediction (y=x)")
plt.xlabel("True Ratings")
plt.ylabel("Predicted Ratings")
plt.title("Fine-Tuning BERT Predictions")
plt.legend()
plt.show()


y_test_numpy = y_test_tensor.numpy()
y_pred_knn_numpy = y_pred_knn
y_pred_nn_numpy = y_pred_nn.numpy()
y_pred_finetune_numpy = y_pred


# 5. Comparación KNN, NN y Fine-Tuning BERT
plt.figure(figsize=(10, 6))
plt.scatter(y_test_numpy, y_pred_knn_numpy, color='blue', alpha=0.6, label="KNN Predictions")
plt.scatter(y_test_numpy, y_pred_nn_numpy, color='red', alpha=0.6, label="NN Predictions")
plt.scatter(y_test_numpy, y_pred_finetune_numpy, color='green', alpha=0.6, label="Fine-Tuning BERT Predictions")
plt.plot([y_test_numpy.min(), y_test_numpy.max()], [y_test_numpy.min(), y_test_numpy.max()], color='black', linestyle='--', label="Perfect Prediction (y=x)")
plt.xlabel("True Ratings")
plt.ylabel("Predicted Ratings")
plt.title("Comparison: KNN vs NN vs Fine-Tuning BERT")
plt.legend()
plt.show()

############################################################################
####################### EXTENSION ###########################################
######################################################################
from transformers import pipeline
import pandas as pd


# Cargar los pipelines para resumen con tres modelos diferentes
summarizer_bart = pipeline("summarization", model="facebook/bart-large-cnn")
summarizer_t5 = pipeline("summarization", model="t5-small")
summarizer_pegasus = pipeline("summarization", model="google/pegasus-xsum")

# Funciones para resumir las instrucciones
def summarize_instructions_bart(text):
    input_length = len(text.split())
    max_length = min(150, input_length)  # Ajustar max_length según el tamaño del texto
    summary = summarizer_bart(text, max_length=max_length, min_length=50, do_sample=False)
    return summary[0]['summary_text']

def summarize_instructions_t5(text):
    input_length = len(text.split())
    max_length = min(150, input_length)  # Ajustar max_length según el tamaño del texto
    summary = summarizer_t5(text, max_length=max_length, min_length=50, do_sample=False)
    return summary[0]['summary_text']

def summarize_instructions_pegasus(text):
    input_length = len(text.split())
    max_length = min(150, input_length)  # Ajustar max_length según el tamaño del texto
    summary = summarizer_pegasus(text, max_length=max_length, min_length=50, do_sample=False)
    return summary[0]['summary_text']

recipes_df = recipes_df.head(3)

# Resumir las instrucciones para todas las filas del DataFrame
results = []
for index, row in recipes_df.iterrows():
    recipe_directions = row["directions"]

    # Generar resúmenes con cada modelo
    summary_bart = summarize_instructions_bart(' '.join(recipe_directions))
    summary_t5 = summarize_instructions_t5(' '.join(recipe_directions))
    summary_pegasus = summarize_instructions_pegasus(' '.join(recipe_directions))

    results.append({
        'index': index,
        'original': ' '.join(recipe_directions),
        'summary_bart': summary_bart,
        'summary_t5': summary_t5,
        'summary_pegasus': summary_pegasus
    })


# Convertir los resultados a un DataFrame
summary_df = pd.DataFrame(results)


# Mostrar los resultados
print(summary_df)





